<a href="https://colab.research.google.com/github/rcpsilva/BCC342_Intro_to_Optimization/blob/main/Knapsack_PythonMIP_Greedy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mip

In [2]:
from mip import Model, xsum, maximize, BINARY
import numpy as np

In [3]:
p = [45,35,18,4,10,2]
w = [100,50,45,20,10,5]
c = 100
n = len(p)

In [4]:
m = Model('Knapsack')

In [5]:
x = [m.add_var(var_type=BINARY) for i in range(n)]

m.objective = maximize(xsum(x[i]*p[i] for i in range(n)))

m += xsum(x[i]*w[i] for i in range(n)) <= c

In [6]:
m.optimize()

<OptimizationStatus.OPTIMAL: 0>

In [7]:
print([x[i].x for i in range(n)])

[0.0, 1.0, 1.0, 0.0, 0.0, 1.0]


In [8]:
selected = [i for i in range(n) if x[i].x >= 0.99]
selected

[1, 2, 5]

In [9]:
def brute_force(fobj,n,m,partial_solution=[],best_solution=[],best_val=np.inf,print_sol=False):

  if len(partial_solution) == n:
    
    fx = fobj(partial_solution)
    
    if print_sol:
      print('{} : {}'.format(partial_solution,fx))

    if fx <= best_val:
      best_solution = partial_solution
      best_val = fx

    return best_solution,best_val
  
  else:
    for e in np.arange(m):
      best_solution,best_val = brute_force(fobj,n,m,partial_solution + [e],
                best_solution,
                best_val,
                print_sol)
    
    return best_solution,best_val

In [11]:
fobj = lambda x : -np.sum([x[i]*p[i] for i in range(n)]) if np.sum([x[i]*w[i] for i in range(n)]) <= c else np.inf

In [14]:
best_solution,best_val = brute_force(fobj,n,2,print_sol=False)
best_solution,best_val

([0, 1, 1, 0, 0, 1], -55)

In [15]:
def greedy_knapsack(h,n,c):
  sol = [0 for i in range(n)]
  weight = 0

  hs = [h(e) for e in range(n)]

  i = 0

  while i < n:
    item = np.argmax(hs)

    weight += w[item]

    if weight > c:
      hs[item] = -np.inf
      weight -= w[item]
    else:
      sol[item] = 1
      hs[item] = -np.inf
    
    i += 1

  return sol

In [23]:
h = lambda e: p[e]
sol = greedy_knapsack(h,n,c)
print('{} P:{}'.format(sol,-fobj(sol)))

[1, 0, 0, 0, 0, 0] P:45


In [22]:
h = lambda e: -w[e]
sol = greedy_knapsack(h,n,c)
print('{} P:{}'.format(sol,-fobj(sol)))

[0, 0, 1, 1, 1, 1] P:34


In [26]:
h = lambda e: -w[e]*0.2 + p[e]*0.8
sol = greedy_knapsack(h,n,c)
print('{} P:{}'.format(sol,-fobj(sol)))

[0, 1, 0, 1, 1, 1] P:51


In [25]:
h = lambda e: p[e]/w[e] 
sol = greedy_knapsack(h,n,c)
print('{} P:{}'.format(sol,-fobj(sol)))

[0, 1, 0, 1, 1, 1] P:51


In [31]:
def grasp_knapsack(h,n,c,prob=0.4):
  sol = [0 for i in range(n)]
  weight = 0

  hs = [h(e) for e in range(n)]

  i = 0

  while i < n:
    if np.random.rand() <= prob:
      item = np.argmax(hs)
    else:
      item = np.random.randint(n)

    weight += w[item]

    if weight > c:
      hs[item] = -np.inf
      weight -= w[item]
    else:
      sol[item] = 1
      hs[item] = -np.inf
    
    i += 1

  return sol

In [53]:
h = lambda e: p[e]/w[e] 
sol = grasp_knapsack(h,n,c,.5)
print('{} P:{}'.format(sol,-fobj(sol)))

[0, 1, 0, 0, 1, 1] P:47
